In [ ]:
#%% Importing the libraries

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [ ]:
#%% Importing the dataset

# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [ ]:
#%% Preparing the training set and the test set

training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t', header = None)
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t', header = None)
test_set = np.array(test_set, dtype = 'int')

In [ ]:
#%% Getting the number of users and movies

nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

mins = int(min(min(training_set[:, 1], ), min(test_set[:, 1])))

In [ ]:
#%% Converting the data into an array with users in lines and movies in columns

def convertData(data:np.array):
    """ 
        Convert training data into 2D matrix where row indicies equal users and
        columns equal movie id. Cell reffers to user rating.
        
        Fill 0 for user who did not rate a movie.
        
        Information from training data:
        Column 1: User -> min user = 1, max user = 943
        Column 2: Movie -> min id = 1, max id = 1682
        Column 3: Rating
        Column 4: Timestamp
    
    """
    convert_data = []
    for user in range(1, nb_users + 1):
        zeros = [0] * (nb_movies + 1)
        single_user_reviews = data[data[:, 0] == user]
        for rating in single_user_reviews:
            zeros[rating[1]] = rating[2]
        zeros.pop(0) #no moview id that = 0
        convert_data.append(zeros)
    
    return convert_data

training_set = convertData(training_set)
test_set = convertData(test_set)

In [ ]:
#%% Converting the data into Torch tensors

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [ ]:
#%% Creating the architecture of the Neural Network

class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
       x = self.activation(self.fc1(x))
       x = self.activation(self.fc2(x))
       x = self.activation(self.fc3(x))
       x = self.fc4(x)    
       return x
   
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay= 0.5)   

In [ ]:
#%% Training the SAE

nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input_x = Variable(training_set[id_user]).unsqueeze(0)
        target = input_x.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae.forward(input_x)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    
    print('epoch: ' + str(epoch) + ' loss: '+ str(train_loss/s))

epoch: 1 loss: 1.7710038021536338
epoch: 2 loss: 1.0967942641436759
epoch: 3 loss: 1.0534919115681054
epoch: 4 loss: 1.038341413930447
epoch: 5 loss: 1.0307965015720548
epoch: 6 loss: 1.0266279998988606
epoch: 7 loss: 1.0237359782155178
epoch: 8 loss: 1.022003730726132
epoch: 9 loss: 1.0211101033059193
epoch: 10 loss: 1.0198268054351565
epoch: 11 loss: 1.0189284032221775
epoch: 12 loss: 1.0182938190793258
epoch: 13 loss: 1.0178759194903797
epoch: 14 loss: 1.0176890740380709
epoch: 15 loss: 1.0173441093582898
epoch: 16 loss: 1.0168743334327013
epoch: 17 loss: 1.0168435134454028
epoch: 18 loss: 1.0165272311725169
epoch: 19 loss: 1.0164157870613109
epoch: 20 loss: 1.016310107251001
epoch: 21 loss: 1.0157790972286351
epoch: 22 loss: 1.0163073883870248
epoch: 23 loss: 1.015571700391488
epoch: 24 loss: 1.0159257136274675
epoch: 25 loss: 1.0157679560151942
epoch: 26 loss: 1.0158646468338834
epoch: 27 loss: 1.0150759930836835
epoch: 28 loss: 1.0152429804774221
epoch: 29 loss: 1.013872872582669

In [ ]:
#%% Testing the SAE

test_loss = 0
s = 0.
for id_user in range(nb_users):
    input_x = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input_x)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.

print('test loss: '+ str(test_loss/s))

test loss: 0.9482630600945305

